# Amazon SageMaker Model - Bias Monitor


## Section 1 - Setup <a id='setup'></a>

#### 1.1 Import necessary libraries

In [1]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import io
import random
import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

from threading import Timer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
CPU times: user 1.26 s, sys: 182 ms, total: 1.44 s
Wall time: 1.62 s


#### 1.2 AWS region and IAM Role & Sagemaker Clients and Bucket Details

In [2]:
sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

# Retrieve the default Amazon S3 bucket associated with the SageMaker session.
bucket = sagemaker_session.default_bucket()
print("Bucket:", bucket)

# Get the IAM role associated with the current SageMaker notebook or environment.
role = get_execution_role()
print("RoleArn:", role)

# Get the AWS region name for the current session.
region = boto3.Session().region_name
print("Region", region)

# Retrieve the AWS account ID of the caller using the Security Token Service (STS) client.
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Boto3 client for the SageMaker service, specifying the AWS region.
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Create an S3 client
s3 = boto3.client('s3')

Bucket: sagemaker-us-east-1-084375567266
RoleArn: arn:aws:iam::084375567266:role/service-role/SageMaker-ExecutionRole-20250222T162355
Region us-east-1


In [3]:
endpoint_name = "retain-ai-xgb-endpoint"
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
display(response)

{'EndpointName': 'retain-ai-xgb-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:084375567266:endpoint/retain-ai-xgb-endpoint',
 'EndpointConfigName': 'retain-ai-xgb-endpoint',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost@sha256:4814427c3e0a6cf99e637704da3ada04219ac7cd5727ff62284153761d36d7d3',
     'ResolutionTime': datetime.datetime(2025, 2, 23, 20, 24, 57, 274000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'DataCaptureConfig': {'EnableCapture': True,
  'CaptureStatus': 'Started',
  'CurrentSamplingPercentage': 100,
  'DestinationS3Uri': 's3://sagemaker-us-east-1-203012117619/aai-540-group-3-final-project/data/monitoring'},
 'EndpointStatus': 'InService',
 'CreationTime': da

In [4]:
# Download the file from S3 to a local file object
houldout_file_key = "aai-540-group-3-final-project/data/holdout/holdout.csv"
response = s3.get_object(Bucket=bucket, Key=houldout_file_key)

# Read the content of the file into a pandas DataFrame
data_df = pd.read_csv(response['Body'])

# Display the DataFrame
display(data_df)

all_headers = data_df.columns.to_list()
print(all_headers)

label_header = all_headers[len(all_headers) - 1]
print(label_header)

,Employee ID,Age,Gender,Years at Company,Job Role,Monthly Income,Work-Life Balance,Job Satisfaction,Performance Rating,Number of Promotions,...,Number of Dependents,Job Level,Company Size,Company Tenure,Remote Work,Leadership Opportunities,Innovation Opportunities,Company Reputation,Employee Recognition,Attrition
0,44132,38,1,23,1,10351,0,2,1,2,...,4,0,2,54,0,0,0,3,2,1
1,67355,22,1,13,2,8012,1,0,2,0,...,1,2,1,74,0,0,0,2,0,1
2,67290,40,0,32,4,6157,1,1,1,0,...,4,1,2,62,0,0,0,3,1,0
3,25581,33,0,8,2,9281,2,0,0,2,...,0,0,1,25,0,0,0,2,1,0
4,39986,57,1,22,3,6116,3,0,0,2,...,2,1,1,38,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29794,37472,32,1,13,2,8809,1,2,0,2,...,4,1,1,26,0,0,0,2,2,1
29795,16814,45,0,16,4,9907,2,0,0,0,...,1,0,2,79,1,0,0,3,2,0
29796,15541,28,1,3,3,5238,0,0,3,0,...,1,1,1,5,0,0,0,2,0,0
29797,56431,39,1,2,1,4814,2,2,0,2,...,2,1,1,74,0,1,0,2,1,0


['Employee ID', 'Age', 'Gender', 'Years at Company', 'Job Role', 'Monthly Income', 'Work-Life Balance', 'Job Satisfaction', 'Performance Rating', 'Number of Promotions', 'Overtime', 'Distance from Home', 'Education Level', 'Marital Status', 'Number of Dependents', 'Job Level', 'Company Size', 'Company Tenure', 'Remote Work', 'Leadership Opportunities', 'Innovation Opportunities', 'Company Reputation', 'Employee Recognition', 'Attrition']
Attrition


#### 1.3 S3 bucket and prefixes

In [5]:
# Bucket prefix to store details for the monitor
prefix = "sagemaker/clarify-bias-monitor"

# Other prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-084375567266/sagemaker/clarify-bias-monitor/datacapture
Ground truth path: s3://sagemaker-us-east-1-084375567266/sagemaker/clarify-bias-monitor/ground_truth_data/2025-02-23-21-54-10
Report path: s3://sagemaker-us-east-1-084375567266/sagemaker/clarify-bias-monitor/reports


## 2. Call Endpoint with Houldout Data & Upload Ground Truth

In [6]:
import matplotlib.pyplot as plt

def print_predictions_graphically(predictions):
    """
    Print predictions graphically as dots and dashes.

    Parameters:
    - predictions (list): List of predictions (either probabilities or class labels).
    """
    for prediction in predictions:
        try:
            # Convert prediction to float if it's not already
            pred_value = float(prediction)

            # Print a dot for values >= 0.5, and a dash for values < 0.5
            if pred_value >= 0.5:
                print(".", end="", flush=True)
            else:
                print("-", end="", flush=True)

        except ValueError:
            print("Invalid prediction value:", prediction)
    
    print()  # Move to the next line after all predictions are printed


In [7]:
display(len(data_df))

29799

In [8]:
# To store predictions and their corresponding employee IDs
predictions_store = []
employee_ids_store = []

# Flag to stop the scheduler
stop_scheduler = False

# Function to invoke SageMaker endpoint and track predictions
def invoke_sagemaker_endpoint(data_df=data_df, num_rows=20, endpoint_name="retain-ai-xgb-endpoint"):
    """
    Prepare data and send it to a SageMaker endpoint for prediction.
    """
    # Select random rows
    rows = random.sample(range(len(data_df)), num_rows)
    selected_data = data_df.iloc[rows]
    
    # Drop the 'Attrition' column as it's the target variable
    input_data = selected_data.drop(columns=['Attrition'])
    
    # Create a SageMaker runtime client
    runtime_client = boto3.client('sagemaker-runtime')

    # Loop through the selected rows
    predictions = []
    employee_ids = []

    for index, row in input_data.iterrows():
        row_data = row.to_frame().T  # Convert the row to a DataFrame (single-row)
        
        # Convert to CSV format without the header
        csv_buffer = io.StringIO()
        row_data.to_csv(csv_buffer, index=False, header=False)
        row_payload = csv_buffer.getvalue()  # Get the CSV payload for the row
        
        # Invoke the endpoint for the row
        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',  # Assuming the model expects CSV input
            Body=row_payload
        )
        
        # Get the prediction from the response
        prediction = response['Body'].read().decode('utf-8')
        predictions.append(prediction)
        
        # Store the Employee ID for the prediction
        employee_ids.append(row['Employee ID'])

    print_predictions_graphically(predictions)
    # Store predictions and IDs for later ground truth upload
    predictions_store.append(predictions)
    employee_ids_store.append(employee_ids)

In [10]:
# Function to upload ground truth every hour
def upload_ground_truth(upload_time):
    # Get the current time for timestamping the folder in S3
    upload_time = datetime.utcnow() - timedelta(hours=1)

    # Flatten the stored Employee IDs into a list of ground truth records
    ground_truth_records = []
    for ids in employee_ids_store:  # Only need employee_ids for actual labels
        for emp_id in ids:  # For each Employee ID
            # Fetch the actual Attrition label from data_df for the given Employee ID
            actual_attrition = data_df.loc[data_df['Employee ID'] == emp_id, 'Attrition'].values[0]

            # Now use the actual Attrition label (not the predicted one) for ground truth
            ground_truth_records.append({
                "groundTruthData": {
                    "data": str(actual_attrition),  # Use the actual Attrition value (1 or 0)
                    "encoding": "CSV",
                },
                "eventMetadata": {
                    "eventId": str(emp_id),  # Use Employee ID as the unique identifier
                },
                "eventVersion": "0",
            })

    # Convert ground truth records to JSONL (newline-delimited JSON)
    upload_records = [json.dumps(record) for record in ground_truth_records]
    data_to_upload = "\n".join(upload_records)
    
    # Upload to S3
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(upload_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)
   
    # Clear the stored Employee IDs after uploading
    employee_ids_store.clear()

In [11]:
# Schedule the prediction task every minute
def schedule_predictions():
    invoke_sagemaker_endpoint()
    if stop_scheduler:
        print("Scheduler stopped.")
        return 
    Timer(10, schedule_predictions).start()  # Reschedule to run every 10s

# Start the prediction scheduling
schedule_predictions()

..--.-...---..--....


In [12]:
# Start the upload function for the first time
upload_ground_truth(datetime.utcnow() - timedelta(hours=1))

Uploading 20 records to s3://sagemaker-us-east-1-084375567266/sagemaker/clarify-bias-monitor/ground_truth_data/2025-02-23-21-54-10/2025/02/23/20/5523.jsonl
....-...-.-..--.-.--


In [13]:
# Reschedule the function to run every hour with the current time as an argument
def schedule_upload():
    if stop_scheduler:
        print("Scheduler stopped.")
        return
    # Get current time for timestamping the folder in S3
    upload_time = datetime.datetime.now()
    
    # Use lambda to pass the upload_time argument to the function
    Timer(60, lambda: upload_ground_truth(upload_time)).start()

....-..----.-.....--
-.---.--.-......-...


In [ ]:
# To stop the scheduler at any point, set the flag to True
# Example: to stop the schedule, call the following line
stop_scheduler = True
# Clear the stored Employee IDs after uploading
employee_ids_store.clear()

## 3 Setup Bias Monitor

In [15]:
model_bias_monitor = ModelBiasMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

model_bias_config = BiasConfig(
    label_values_or_threshold=[1],
    facet_name="Job Satisfaction", # sensitive feature Job Satisfaction to check for bias
    facet_values_or_threshold=[0, 1, 2, 3],  # Monitoring bias for all categories of Job Satisfaction
)

model_bias_analysis_config = BiasAnalysisConfig(
    model_bias_config,
    headers=all_headers, # List of header names for your input data
    label=label_header, # The column name for the label (e.g., Attrition)
)

## 3 Setup Monitor Schedule

In [ ]:
# If needed, delete existing monitoring schedule
model_bias_monitor.delete_monitoring_schedule()

In [16]:
# Define cron expression for scheduling monitoring job every hour
schedule_expression = CronExpressionGenerator.hourly()

# Set up the monitoring schedule
model_bias_monitor.create_monitoring_schedule(
    analysis_config=model_bias_analysis_config,
    output_s3_uri=s3_report_path,  # Specify where to save the results (S3 path)
    endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint",  # Input destination directory for the endpoint data
        start_time_offset="-PT1H",  # Start 1 hour before now
        end_time_offset="-PT0H",  # End at the current time
        probability_threshold_attribute=0.8,  # Only high-confidence predictions (with a probability ≥ 0.8) are considered for further analysis, including bias detection 
    ),
    ground_truth_input=ground_truth_upload_path,  # S3 path for the ground truth data
    schedule_cron_expression=schedule_expression,  # Schedule the monitoring to run hourly
)

print(f"Model bias monitoring schedule: {model_bias_monitor.monitoring_schedule_name}")


Model bias monitoring schedule: monitoring-schedule-2025-02-23-21-56-05-854


In [17]:
# restart schedule if needed 

# model_bias_monitor.stop_monitoring_schedule()

model_bias_monitor.start_monitoring_schedule()

-.......-.----.---.-


In [ ]:
# Check for start 
def wait_for_execution_to_start(model_monitor):
    print(
        "A hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer)."
    )

    print("Waiting for the first execution to happen", end="")
    schedule_desc = model_monitor.describe_schedule()
    while "LastMonitoringExecutionSummary" not in schedule_desc:
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        sleep(60)
    print()
    print("Done! Execution has been created")

    print("Now waiting for execution to start", end="")
    while schedule_desc["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"] in "Pending":
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        sleep(10)

    print()
    print("Done! Execution has started")
        
wait_for_execution_to_start(model_bias_monitor)

A hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer).
Waiting for the first execution to happen.....-----.---.-.-.-.
..-...--.--....-.--.
.--.--.---...--..-.-
-....-.--.-..--.---.
-----..--.---.-...-.
....----..-..--.....
......-..-.----.-...-
-..---.-..--.....-..
...-.-..-..-.---....
.-.....-.---------..
-..-.---..--..--.-.-
---.-..--.-.-.--..-.
..-..--.--.-.-.----.-
----.--...--.-....--
...----...-.---.---.
----.....--...-.....
--....-.---.-..-...-
.-..-....-.--.--....
.-.-.-.-....----.-.--
.....----..-........
-....---.-...---..--
...------.--....----
----...-..-.------..
-........---...--..-
.---.-.......-.-...-.
--..---..-..-.....--
-.-----.----------.-


In [66]:
# Waits for the schedule to have last execution in a terminal status.
def wait_for_execution_to_finish(model_monitor):
    schedule_desc = model_monitor.describe_schedule()
    execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
    if execution_summary is not None:
        print("Waiting for execution to finish", end="")
        while execution_summary["MonitoringExecutionStatus"] not in [
            "Completed",
            "CompletedWithViolations",
            "Failed",
            "Stopped",
        ]:
            print(".", end="", flush=True)
            sleep(60)
            schedule_desc = model_monitor.describe_schedule()
            execution_summary = schedule_desc["LastMonitoringExecutionSummary"]
        print()
        print("Done! Execution has finished")
    else:
        print("Last execution not found")
            
wait_for_execution_to_finish(model_bias_monitor)

Waiting for execution to finish
Done! Execution has finished


In [67]:
schedule_desc = model_bias_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")

display(execution_summary)

{'MonitoringScheduleName': 'monitoring-schedule-2025-02-23-04-37-13-497',
 'ScheduledTime': datetime.datetime(2025, 2, 23, 6, 0, tzinfo=tzlocal()),
 'CreationTime': datetime.datetime(2025, 2, 23, 6, 2, 20, 261000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 2, 23, 6, 8, 1, 220000, tzinfo=tzlocal()),
 'MonitoringExecutionStatus': 'Failed',
 'EndpointName': 'retain-ai-xgb-endpoint',
 'FailureReason': 'No S3 objects found under S3 URL "s3://sagemaker-us-east-1-084375567266/sagemaker/clarify-bias-monitor/ground_truth_data/ground_truth.csv/2025/02/23/05" given in input data source. Please ensure that the bucket exists in the selected region (us-east-1), that objects exist under that S3 prefix, and that the role "arn:aws:iam::084375567266:role/service-role/SageMaker-ExecutionRole-20250222T162355" has "s3:ListBucket" permissions on bucket "sagemaker-us-east-1-084375567266".'}

## Check Results

In [42]:
schedule_desc = model_bias_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
if execution_summary and execution_summary["MonitoringExecutionStatus"] in [
    "Completed",
    "CompletedWithViolations",
]:
    last_model_bias_monitor_execution = model_bias_monitor.list_executions()[-1]
    last_model_bias_monitor_execution_report_uri = (
        last_model_bias_monitor_execution.output.destination
    )
    print(f"Report URI: {last_model_bias_monitor_execution_report_uri}")
    last_model_bias_monitor_execution_report_files = sorted(
        S3Downloader.list(last_model_bias_monitor_execution_report_uri)
    )
    print("Found Report Files:")
    print("\n ".join(last_model_bias_monitor_execution_report_files))
else:
    last_model_bias_monitor_execution = None
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

====STOP==== 
 No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.


## Clean up <a id='cleanup'></a>  

In [68]:
invoke_endpoint_thread.terminate()

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [69]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name, sagemaker_session=sagemaker_session)
model_monitors = predictor.list_monitors()
for model_monitor in model_monitors:
    model_monitor.stop_monitoring_schedule()
    model_monitor.delete_monitoring_schedule()

In [70]:
predictor.delete_endpoint()
predictor.delete_model()

ClientError: An error occurred (ValidationException) when calling the DescribeEndpointConfig operation: Could not find endpoint configuration "retain-ai-xgb-endpoint".

## References

https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/fairness_and_explainability/SageMaker-Model-Monitor-Fairness-and-Explainability.html